<a href="https://colab.research.google.com/github/fahrilAlfarizi/Citra-Digital/blob/main/Klasifikasi-wajah_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fahrilsidikalfarizi","key":"55180c8310e711fc39018503bde5d5c5"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d niten19/face-shape-dataset

rm: cannot remove '/root/.kaggle': No such file or directory
 98% 626M/639M [00:05<00:00, 142MB/s]
100% 639M/639M [00:05<00:00, 124MB/s]


In [ ]:
!mkdir Dataset
!cp /content/face-shape-dataset.zip /content/face-shape-dataset.zip
!unzip -q /content/face-shape-dataset.zip -d /content/Dataset
!rm /content/face-shape-dataset.zip

cp: '/content/face-shape-dataset.zip' and '/content/face-shape-dataset.zip' are the same file


In [ ]:
import os
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Fungsi untuk membaca dan mengekstrak fitur dari gambar
def extract_features_from_folder(folder_path, label, localized_file_names, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            # Ubah ukuran gambar ke target_size
            img = cv2.resize(img, target_size)
            flattened_img = img.flatten()
            images.append(flattened_img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Path ke folder dataset
dataset_folder = '/content/Dataset/FaceShape Dataset'  # Ganti dengan path yang sesuai

# List nama-nama class (bentuk wajah)
classes = ['Heart', 'Oblong', 'Oval', 'Round', 'Square']

# Inisialisasi struktur data untuk menampung fitur dan label
all_images = []
all_labels = []

In [ ]:
# Loop melalui setiap class dan folder untuk mengambil data
for class_name in classes:
    training_set_path = os.path.join(dataset_folder, 'training_set', class_name)
    testing_set_path = os.path.join(dataset_folder, 'testing_set', class_name)

    if not os.path.exists(training_set_path) or not os.path.exists(testing_set_path):
        print(f"Warning: Folder {class_name} not found in training_set or testing_set.")
        continue

    # Baca data dari folder training_set
    images, labels = extract_features_from_folder(training_set_path, class_name, {})
    all_images.extend(images)
    all_labels.extend(labels)

# Normalisasi data
scaler = StandardScaler()
all_images = scaler.fit_transform(all_images)

# Konversi label menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
all_labels_encoded = label_encoder.fit_transform(all_labels)

# Pisahkan data menjadi training dan testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels_encoded, test_size=0.2, random_state=42)


In [ ]:
# Inisialisasi model KNN
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)  # Sesuaikan jumlah tetangga sesuai kebutuhan

    # Latih model dengan data dan label dari training_set
    knn.fit(X_train, y_train)

# Inisialisasi variabel untuk mengakumulasi akurasi
total_accuracy = 0

# Inisialisasi model KNN
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)  # Sesuaikan jumlah tetangga sesuai kebutuhan

    # Latih model dengan data dan label dari training_set
    knn.fit(X_train, y_train)

    # Evaluasi performa model pada data testing_set
    total_accuracy_k = 0  # Inisialisasi variabel untuk mengakumulasi akurasi pada nilai K tertentu

    for class_name in classes:
        testing_set_path = os.path.join(dataset_folder, 'testing_set', class_name)

        if not os.path.exists(testing_set_path):
            print(f"Warning: Folder {class_name} not found in testing_set.")
            continue

        # Baca data dari folder testing_set
        images_test, labels_test = extract_features_from_folder(testing_set_path, class_name, {})

        # Normalisasi data testing_set
        images_test = scaler.transform(images_test)

        # Konversi label testing_set menjadi angka menggunakan LabelEncoder
        labels_test_encoded = label_encoder.transform(labels_test)

        # Prediksi kelas untuk data testing_set
        predictions = knn.predict(images_test)


        # Evaluasi performa model pada data testing_set
        accuracy = accuracy_score(labels_test_encoded, predictions)
        print(f'K = {k}, Accuracy for {class_name}: {accuracy}')

        # Metrics tambahan (opsional)
        print(f"Classification Report for {class_name}:\n", classification_report(labels_test_encoded, predictions))
        print(f"Confusion Matrix for {class_name}:\n", confusion_matrix(labels_test_encoded, predictions))

    # Hitung akurasi rata-rata pada nilai K tertentu
    avg_accuracy_k = total_accuracy_k / len(classes)

    # Tambahkan ke total akurasi
    total_accuracy += avg_accuracy_k

    # Cetak akurasi pada nilai K tertentu
    print(f'K = {k}, Average Accuracy: {avg_accuracy_k}')

# Hitung dan cetak rata-rata akurasi dari semua nilai K
average_accuracy = total_accuracy # Jumlah iterasi
print(f'Average Accuracy Across All K: {average_accuracy}')

K = 1, Accuracy for Heart: 0.315
Classification Report for Heart:
               precision    recall  f1-score   support

           0       1.00      0.32      0.48       200
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.32       200
   macro avg       0.20      0.06      0.10       200
weighted avg       1.00      0.32      0.48       200

Confusion Matrix for Heart:
 [[63 22 46 32 37]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K = 1, Accuracy for Oblong: 0.275
Classification Report for Oblong:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.28      0.43       200
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.28       200
   macro avg       0.20      0.06      0.09       200
weighted avg       1.00      0.28      0.43       200

Confusion Matrix for Oblong:
 [[ 0  0  0  0  0]
 [38 55 40 26 41]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K = 1, Accuracy for Oval: 0.255
Classification Report for Oval:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.26      0.41       200
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.26       200
   macro avg       0.20      0.05      0.08       200
weighted avg       1.00      0.26      0.41       200

Confusion Matrix for Oval:
 [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [35 42 51 28 44]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K = 20, Accuracy for Round: 0.24
Classification Report for Round:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.24      0.39       200
           4       0.00      0.00      0.00         0

    accuracy                           0.24       200
   macro avg       0.20      0.05      0.08       200
weighted avg       1.00      0.24      0.39       200

Confusion Matrix for Round:
 [[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [26 28 27 48 71]
 [ 0  0  0  0  0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


K = 20, Accuracy for Square: 0.54
Classification Report for Square:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.54      0.70       200

    accuracy                           0.54       200
   macro avg       0.20      0.11      0.14       200
weighted avg       1.00      0.54      0.70       200

Confusion Matrix for Square:
 [[  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [  0   0   0   0   0]
 [ 38  13  28  13 108]]
K = 20, Average Accuracy: 0.0
Average Accuracy Across All K: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
